In [2]:
import requests
import pandas as pd
import time

In [3]:
# Get information about the adress of interest through blockchain API

In [9]:
def create_receiver_list(transaction_ego):
    receiver_addresses = list()
    issues_receiver = list()
    amount = list()
    for j in range(len(transaction_ego)):
        receiver_addresses.append([])
        amount.append([])
        for i in range(len(transaction_ego[j]['out'])):
            try:
                receiver_addresses[-1].append(transaction_ego[j]['out'][i]['addr'])
                amount[-1].append(transaction_ego[j]['out'][i]['value'])
            except:
                issues.append([i,j])
    
    return(receiver_addresses)



def amount_transferred_per_transaction(transaction_ego):
    
    amount = list()
    for j in range(len(transaction_ego)):
        amount.append([])
        for i in range(len(transaction_ego[j]['out'])):
            try:
                amount[-1].append(transaction_ego[j]['out'][i]['value'])
            except:
                 pass
    
    return(amount)




def create_sender_list(transaction_ego):
    
    sender_addresses = list()
    issues_sender = list()
    for j in range(len(transaction_ego)):
        sender_addresses.append([])
        for i in range(len(transaction_ego[j]['inputs'])):
            try:
                sender_addresses[-1].append(transaction_ego[j]['inputs'][i]['prev_out']['addr'])
            except:
                issues.append([i,j])
    return(sender_addresses)

def create_timestamp_list_transaction(transaction_ego):
    return([transaction_ego[i]['time'] for i in range(len(transaction_ego))])

def create_edge_list(sender_addresses, receiver_addresses, amount, timestamps_transactions):
    
    data = pd.DataFrame([], )

    for nested in range(len(sender_addresses)):
        for sender in range(len(sender_addresses[nested])):
            for receiver in range(len(receiver_addresses[nested])):
                data = data.append(pd.DataFrame({'sender': sender_addresses[nested][sender], 'receiver': receiver_addresses[nested][receiver], 'amount': amount[nested][receiver], 'timestamp': timestamps_transactions[nested]}, index=[0]), ignore_index=True)
    return(data)

In [225]:
def get_identifier(hashes, identity):
    '''
    Given a list of senders or receivers and a identity table created with get_identity_table, return the addresses
    mapped with their ID
    '''
    renamed = list()
    for address_list in hashes:
        renamed.append([])
        match = False
        for address in address_list:
            for index_identity in range(len(identity)):
                if address in identity.iloc[index_identity, 0]:
                    renamed[-1].append(identity.iloc[index_identity, 1])
                    match = True

            if match==False:
                renamed[-1].append(address)
    
    return(renamed)

In [126]:
def get_identity_table(senders):
    '''
    Map addresses that appear together as input in at least one transaction to a common ID
    '''
    identity = pd.DataFrame(columns=('address', 'id'))
    id_number = 1

    for nested_list in senders:
        match = False
        for address in nested_list:
            for i in range(len(identity)):
                if address in identity.iloc[i, 0]:
                    identity.iloc[i, 0] = list(set(identity.iloc[i, 0] + nested_list))
                    match = True
            if match==False:
                identity= identity.append({'address':nested_list, 'id':id_number}, ignore_index=True)
                id_number += 1
    
    return(identity)               

In [203]:
def merge_clusters(identity):
    '''
    Merge two rows of the identity table if at least one item is in common
    '''
    i = 0
    j = 0
    while i < len(identity):
        j = 0
        while j < len(identity):
            if (i != j) & len(set(identity.iloc[i,0]).intersection(set(identity.iloc[j,0])))>0:
                identity.iloc[j,0] = identity.iloc[j,0] + identity.iloc[i,0]
                identity.drop(identity.index[i], inplace=True)
                print
            j += 1
        i += 1
    return(identity)

In [ ]:
def get_info_from_address(address):
    r =requests.get('https://blockchain.info/rawaddr/' + address)
    return(r.json())

# Analysis of  Terrorism Fund  Raising
## account 17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD

In [4]:
ego = '17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD'

In [5]:
r =requests.get('https://blockchain.info/rawaddr/17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD')

In [6]:
# represent the inormation in json format
r_js = r.json()

# information about the transactions of this account
transaction_ego = r_js['txs']

In [72]:
senders = create_sender_list(transaction_ego)
senders = [list(set(nested)) for nested in senders]

receivers = create_receiver_list(transaction_ego)
receivers = [list(set(nested)) for nested in receiver]

In [205]:
identity = get_identity_table(senders)
identity = merge_clusters(identity)

In [232]:
from_list = [set(nested) for nested in get_identifier(senders, identity)]
to_list = [set(nested) for nested in get_identifier(receivers, identity)]

### Create an edge list of the transactions of ego with ID instead on names where possible

In [237]:
edges_ego = pd.DataFrame({'from':from_list, 'to':to_list, 'amount':amount_transferred_per_transaction(transaction_ego)})

In [36]:
sender_addresses = create_sender_list(transaction_ego)
receiver_addresses = create_receiver_list(transaction_ego)
timestamps_transactions = create_timestamp_list_transaction(transaction_ego)
amount = amount_transferred_per_transaction(transaction_ego)

In [37]:
data = create_edge_list(sender_addresses, receiver_addresses, amount, timestamps_transactions)

In [38]:
data['datetime'] = [pd.to_datetime(time.strftime('%A, %Y-%m-%d %H:%M:%S', time.localtime(timestamp))) for timestamp in data['timestamp']]

In [42]:
data_ego_2021 = data[data.datetime>=pd.to_datetime('2021-01-01')]

In [43]:
OUT_PUT_PATH = 'edge_folder/edges_2021/ego_edges_2021.csv'
data_ego_2021.to_csv(OUT_PUT_PATH)

In [13]:
len(sender_addresses), len(receiver_addresses), len(timestamps_transactions)

(70, 70, 70)

In [81]:
OUT_PUT_PATH = 'edge_folder/ego_edges1.csv'
data.to_csv(OUT_PUT_PATH)

In [27]:
others = list(set(pd.concat([data.loc[data.datetime>=pd.to_datetime('2021-01-01'), 'receiver'], data.loc[data.datetime>=pd.to_datetime('2021-01-01'), 'sender']]).tolist()))
others.remove(ego)

In [31]:
len(others)

4

In [32]:
def get_edgelist_wallet(address):
    
    r =requests.get('https://blockchain.info/rawaddr/' + address)
    # represent the inormation in json format
    r_js = r.json()

    # information about the transactions of this account
    transaction_ego = r_js['txs']

    sender_addresses = create_sender_list(transaction_ego)
    receiver_addresses = create_receiver_list(transaction_ego)
    timestamps_transactions = create_timestamp_list_transaction(transaction_ego)
    amount = amount_transferred_per_transaction(transaction_ego)

    data = create_edge_list(sender_addresses, receiver_addresses, amount, timestamps_transactions)
    data['datetime'] = [pd.to_datetime(time.strftime('%A, %Y-%m-%d %H:%M:%S', time.localtime(timestamp))) for timestamp in data['timestamp']]
    data = data[data.datetime>=pd.to_datetime('2021-01-01')]
    return(data)

In [35]:
for i in range(len(others)):
    data = get_edgelist_wallet(others[i])
    OUT_PUT_PATH = 'edge_folder/edges_2021/edges_' + others[i] + '.csv'
    data.to_csv(OUT_PUT_PATH)

In [ ]:
import os
import glob

os.chdir("./edge_folder/edges_2021")
file_extension = '.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

In [56]:
file_extension = '.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])
others = list(set(pd.concat([combined_csv_data['receiver'], combined_csv_data['sender']]).tolist()))

OUT_PUT_PATH = './ego_network_2021.csv'
combined_csv_data.to_csv(OUT_PUT_PATH)

In [76]:
others = list(set(pd.concat([combined_csv_data['receiver'], combined_csv_data['sender']]).tolist()))
others = pd.DataFrame({'id':others})
others['label'] = ['target' if target==ego else 'other' for target in others['id'] ]
others[others.id==ego]

,id,label
1748,17QAWGVpFV4gZ25NQug46e5mBho4uDP6MD,target


In [77]:

OUT_PUT_PATH = './nodes_ego_network_2021.csv'
others.to_csv(OUT_PUT_PATH)